In [ ]:
import io
import os
import json
import zipfile
import urllib
from time import strftime, gmtime

import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

bucket = ''

In [ ]:
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
urllib.request.urlretrieve('https://ti.arc.nasa.gov/m/project/prognostic-repository/CMAPSSData.zip', os.path.join(data_folder, 'CMAPSSData.zip'))

with zipfile.ZipFile(os.path.join(data_folder, 'CMAPSSData.zip'), "r") as zip_ref:
    zip_ref.extractall(data_folder)
    
columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3','s4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14','s15', 's16', 's17', 's18', 's19', 's20', 's21']

In [ ]:
# normalize sensor readings
train_df = []
eps = 0.000001 # for floating point issues during normalization 
for i in range(1,5):
    df = pd.read_csv('data/train_FD{:03d}.txt'.format(i), delimiter=' ', header=None)
    df.drop(df.columns[[26, 27]], axis=1, inplace=True)
    df.columns = columns
    df[columns[2:]]=(df[columns[2:]]-df[columns[2:]].min()+eps)/(df[columns[2:]].max()-df[columns[2:]].min()+eps)
    train_df.append(df)

# compute RUL (remaining useful life)
for i, df in enumerate(train_df):
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    df = df.merge(rul, on=['id'], how='left')
    df['RUL'] = df['max'] - df['cycle']
    df.drop('max', axis=1, inplace=True)
    train_df[i]=df

train_df[0].head()
o = train_df[0][columns[2:10]][train_df[0]['id'] == 3].plot(subplots=True, sharex=True, figsize=(20,10), title="Train: 8 sensors of Engine 1 before failure")

In [ ]:
test_df = []
for i in range(1,5):
    # Load time series
    df = pd.read_csv('data/test_FD{:03d}.txt'.format(i), delimiter=' ', header=None)
    df.drop(df.columns[[26, 27]], axis=1, inplace=True)
    
    # Load the RUL values
    df_rul = pd.read_csv('data/RUL_FD{:03d}.txt'.format(i), delimiter=' ', header=None)    
    df_rul.drop(df_rul.columns[1], axis=1, inplace=True)
    df_rul.index += 1
    
    # Merge RUL and timeseries and compute RUL per timestamp
    df = df.merge(df_rul, left_on=df.columns[0], right_index=True, how='left')
    df.columns = columns + ['RUL_end']
    rul = pd.DataFrame(df.groupby('id')['cycle'].max()).reset_index()
    rul.columns = ['id', 'max']
    df = df.merge(rul, on=['id'], how='left') # We get the number of cycles per series
    df['RUL'] = df['max'] + df['RUL_end'] - df['cycle'] # The RUL is the number of cycles per series + RUL - how many cycles have already ran
    df.drop(['max','RUL_end'], axis=1, inplace=True)
    
    # Normalize
    df[columns[2:]]=(df[columns[2:]]-df[columns[2:]].min()+eps)/(df[columns[2:]].max()-df[columns[2:]].min()+eps)
    test_df.append(df)

In [ ]:
import boto3
import os

prefix = 'pred-maintenance-artifacts'

s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

# Upload raw data files to S3
for subdir, dirs, files in os.walk(data_folder):
    for file in files:
        full_path = os.path.join(subdir, file)
        s3_path = os.path.join(prefix, full_path)
        s3_bucket_resource.Object(s3_path).upload_file(full_path)

# Upload processed test data for inference
for i in range(len(test_df)):
    local_test_file = 'data/test-{}.csv'.format(i)
    test_df[i].to_csv(local_test_file)
    s3_test_file = os.path.join(prefix, 'data', 'test-{}.csv'.format(i))
    s3_bucket_resource.Object(s3_test_file).upload_file(local_test_file)

# Upload processed data for training
for i in range(len(train_df)):
    local_train_file = 'data/train-{}.csv'.format(i)
    train_df[i].to_csv(local_train_file)
    s3_train_file = os.path.join(prefix, 'train', 'train-{}.csv'.format(i))
    s3_bucket_resource.Object(s3_train_file).upload_file(local_train_file)

s3_train_data = 's3://{}/{}/{}'.format(bucket, prefix, 'train')
print('uploaded training data location: {}'.format(s3_train_data))

In [ ]:
output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('training artifacts will be uploaded to: {}'.format(output_location))

In [ ]:
from sagemaker.mxnet import MXNet

model_name = "pred-maintenance-mxnet-model"
training_job_name = "{}-{}".format(model_name, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
train_instance_type = 'ml.p3.2xlarge'

m = MXNet(entry_point='entry_point.py',
          source_dir='entry_point',
          py_version='py3',
          role=role, 
          instance_count=1, 
          instance_type=train_instance_type,
          output_path=output_location,
          hyperparameters={'num-datasets' : len(train_df),
                           'num-gpus': 1,
                           'epochs': 500,
                           'optimizer': 'adam',
                           'batch-size':1,
                           'log-interval': 100},
         input_mode='File',
         max_run=7200,
         framework_version='1.6.0')

m.fit({'train': s3_train_data}, job_name=training_job_name)

In [ ]:
batch_output = 's3://{}/{}/{}'.format(bucket, prefix, 'batch-inference')
transformer = m.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

In [ ]:
s3_test_key = "pred-maintenance-artifacts/data/test-0.csv"
s3_transform_input = os.path.join(prefix,  "batch-transform-input")

def get_transform_input():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=s3_test_key)
    test_file = s3_response["Body"].read()

    test_df_entry = pd.read_csv(io.BytesIO(test_file))
    test_data = test_df_entry[test_df_entry['id']==0+1][test_df_entry.columns[2:-1]].values
    test_data = test_data[0:test_data.shape[0]-1,:].astype('float32')
    data_payload = {'input':np.expand_dims(test_data, axis=0).tolist()}
    
    job_name = 'predictive-maintenance-batch-transform-job-{}'.format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))
    s3_batch_transform_input_key = os.path.join(s3_transform_input, job_name)
    
    s3_client.put_object(Body=json.dumps(data_payload),
                         Bucket=bucket, 
                         Key=s3_batch_transform_input_key)
    return job_name, 's3://{}/{}'.format(bucket, s3_batch_transform_input_key)

job_name, input_key = get_transform_input()
transformer.transform(input_key, wait=True)

In [ ]:
def get_transform_output():
    s3_client = boto3.client('s3')
    s3_response = s3_client.get_object(Bucket=bucket, Key=os.path.join(prefix, 
                                                                       'batch-inference', 
                                                                       job_name+'.out'))
    transform_out = np.array(eval(s3_response["Body"].read()))
    return transform_out
    
get_transform_output()